Install libraries

In [ ]:
#Need to install if not installed yet
!pip install matplotlib
!pip install scipy
!pip install sklearn
!pip install pandas
!pip install seaborn
!pip install haversine
!pip install -U wxPython 
!pip install google
!pip install google-api-core
!pip install --upgrade google-api-python-client
!pip install google-cloud
!pip install google-cloud-vision
!pip install google.cloud.bigquery
!pip install google.cloud.storage
!pip install google-auth-oauthlib


Import Libraries

In [1]:
import multiprocessing as mp
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from scipy.stats import uniform
from scipy.stats import norm
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import plot_confusion_matrix
import csv
import pandas as pd
import time
import os
from datetime import datetime, timedelta
import seaborn as sn
import requests
import json
import haversine as hs
import wx
import webbrowser

import DQ2# Own defined

Import Setup file format

In [80]:
## Weights
mu                =0.9
Accuracy          =0.20
Precision         =0.07
Confidence        =0.16
Completeness      =0.10
Timeliness        =0.12
Data_Volume       =0.16
Data_Redundancy   =0.02
Concordance       =0.16

Utility           =0.12
Accessibility     =0.16
Interpretability  =0.28
Reputation        =0.12
Artificiality     =0.20
Access_Security   =0.12


#Period
start_time ="2020-02-01 00:00:00"
end_time   ="2020-02-29 23:59:00"

#Variable Inicialization
node=1


The steps should be like:

0. Clean the whole dataset (the variables of interest):DONE
1. For each citizen science (CC) node, get the groups (HOURLY GROUPS).
2. For each group (hour in a CC node data), calculate the Dimension's DQ. (The functions should be applied to each group instead)
3. Save the result file in the form: Node, Group (hour), DQ_1, DQ_2, DQ3, ... , DQIndex  (This is new)
4. Average the previous result over the whole time to get Node, DQ_1_time_mean, DQ_2_time_mean, DQ3_ave, ... , DQIndex_time_mean (This it what we have currently)
5. Average the previous result over all the nodes to get DQ_1_node_mean, DQ_2_node_mean, DQ3_node_mean, ... , DQIndex_node_mean (This is new)
6. Export 3, 4 and 5 to a Google Sheets page each one.

Read Data

In [2]:
try:
    #Read Data from February
    header_CC=["codigoSerial", "fecha", "hora", "fechaHora", "temperatura", "humedad_relativa", "pm1_df", "pm10_df", "pm25_df", "pm1_nova", "pm10_nova", "pm25_nova", "calidad_temperatura", "calidad_humedad_relativa", "calidad_pm1_df", "calidad_pm10_df", "calidad_pm25_df", "calidad_pm1_nova", "calidad_pm10_nova", "calidad_pm25_nova"]
    datatypes_CC={"codigoSerial":np.uint16, "temperatura":np.float16, "humedad_relativa":np.float16, "pm1_df":np.float32, "pm10_df":np.float32, "pm25_df":np.float32, "pm1_nova":np.float32, "pm10_nova":np.float32, "pm25_nova":np.float32}
    path_for_CC_data=get_path('*.csv',"Select Citizen Scientist *.csv file")
    df_CC = pd.read_csv(path_for_CC_data, header=None, names=header_CC, usecols=header_CC , dtype=datatypes_CC,parse_dates=["fecha","hora","fechaHora"])
    df_CC.sort_values(by=['codigoSerial','fechaHora'],ignore_index=True)
    print("Source path for Citizen Science nodes data: ",path_for_CC_data)

    #Data includes January, February and March
    header_SS=["Fecha_Hora","codigoSerial","pm25","calidad_pm25","pm10","calidad_pm10"]
    datatypes_SS={"codigoSerial":np.uint16,"pm25":np.float32,"pm10":np.float32}
    path_for_SS_data=get_path('*.csv',"Select SIATA Stations *.csv file")
    df_SS = pd.read_csv(path_for_SS_data, header=None,names=header_SS, usecols=header_SS , dtype=datatypes_SS,parse_dates=["Fecha_Hora"])
    df_SS.sort_values(by=['codigoSerial','Fecha_Hora'],ignore_index=True)
    print("Source path for Siata Stations data: ",path_for_SS_data)
        
except:
    print("An exception occurred, it is possible that wrong files were chosen, please run again")

datatypesDistances={"codigoSerial_CC":np.uint16,"codigoSerial_ES":np.uint16,"Distancia_a_ES":np.float16,"codigoSerial_ES2":np.uint16}
Distances = pd.read_csv("C:/Users/julio/Documents/UDEA/Maestría/DQ in IOT/Datasets/Distances and positions/Distancias_2.csv", header=0, dtype=datatypesDistances,index_col="codigoSerial_CC")

#DATA CLEANING
CC, SS=clean_sort_data(df_CC, df_SS)
del df_CC
del df_SS


Citizen Scientist:  [1, 2, 3, 4, 5, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 94, 95, 96, 97, 99, 100, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 127, 128, 129, 130, 131, 133, 134, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 149, 150, 151, 152, 153, 154, 156, 157, 159, 160, 161, 162, 163, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 178, 179, 180, 181, 182, 184, 185, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 213, 214, 215, 216, 217, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 240, 241, 242, 243

Main Code with parallelization

In [44]:
t0= time.time()
print("Start time: ", datetime.fromtimestamp(t0))


dim_time = pd.DataFrame(
        columns =["codigoSerial",
                  "fechaHora",
                  "precision_df_time",
                  "precision_nova_time",
                  "uncertainty_time",
                  "accuracy_df_time",
                  "accuracy_nova_time",
                  "completeness_df_time",
                  "completeness_nova_time",
                  "concordance_df_nova_time",
                  "concordance_df_siata",#MAYBE NEED TO BE CALCULATED ON A DAILY BASIS
                  "concordance_df_hum_time",
                  "concordance_df_temp_time",
                  "concordance_nova_siata",#MAYBE NEED TO BE CALCULATED ON A DAILY BASIS
                  "concordance_nova_hum_time",
                  "concordance_nova_temp_time"])

if __name__ ==  '__main__':
    print("Number of avaliable CPUs: ",mp.cpu_count())
    pool=mp.Pool(processes = mp.cpu_count())
    arguments=[]
    #results=pool.map(DQ.eval_dq,[nodes for nodes in CC.keys()])
    results=pool.map(DQ2.eval_dq,([[nodes, CC, SS, Distances, start_time, end_time] for nodes in CC.keys()]))
    
    #results[10]

    for i in range(0,len(results)):
        dim_time=dim_time.append(results[i], ignore_index = True)
    
print("End Time: ", datetime.fromtimestamp(time.time()))
t1 = time.time() - t0
print("Elapsed Time: ", t1)



Start time:  2021-09-27 23:13:29.733153
Number of avaliables CPUs:  6
End Time:  2021-09-27 23:20:38.388171
Elapsed Time:  428.67064237594604


Export to Google Sheets

In [45]:
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import pandas as pd

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1gkU9I2EqgJCLm-XemWq_oFenkOcsaHRKwDpO2kK7vd8'
#SAMPLE_RANGE_NAME = 'Class Data!A2:E'

def main():
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    
    dim_time.replace(np.nan, '', inplace=True)
    dim_time['fechaHora'] = dim_time['fechaHora'].astype(str)
    
    print("Clearing the page in the google Spread sheet")
    sheet = service.spreadsheets()
    #request = service.spreadsheets().values().clear(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=range_, body=clear_values_request_body)
    result = sheet.values().clear(
        spreadsheetId=SAMPLE_SPREADSHEET_ID,
        range='DQ_TIME!A1:Z1000000',
        
    ).execute()
    
    print("Exporting Data to Google Sheets")
    sheet = service.spreadsheets()
    result = sheet.values().update(
        spreadsheetId=SAMPLE_SPREADSHEET_ID,
        valueInputOption='RAW',
        range='DQ_TIME!A1',
        body=dict(
            majorDimension='ROWS',
            values=dim_time.T.reset_index().T.values.tolist())
    ).execute()
    print("Exporting Data Finished")
#
if __name__ == '__main__':
    main()
    

{'spreadsheetId': '1gkU9I2EqgJCLm-XemWq_oFenkOcsaHRKwDpO2kK7vd8', 'clearedRange': 'DQ_TIME!A1:Z60470'}


To Open the Spreadsheet

In [2]:
webbrowser.open('https://docs.google.com/spreadsheets/d/1gkU9I2EqgJCLm-XemWq_oFenkOcsaHRKwDpO2kK7vd8/edit?usp=sharing')

True

To Open the Report

In [3]:
webbrowser.open('https://datastudio.google.com/s/ietWLq_iL-8')

True